<center>
<h4>Diplomatura AACSyA 2018 - FaMAF - UNC</h4>
<h3>Aprendizaje Supervisado</h3>
<hr>
<h4>Por David Gonzalez <leonardo.david.gonzalez@gmail.com> y Facundo Díaz Cobos <facundo.diaz.cobos@gmail.com></h4>
</center>
<br/>

# Aprendizaje Supervisado: Laboratorio 2 - Iteracion 2


Movies Reviews Sentiment Analysis

    Trabajo final de laboratorio de la materia Aprendizaje Supervisado (Matías Marenchino, Franco M. Luque)
    Diplomatura en Ciencias de Datos, Aprendizaje Automático y sus Aplicaciones http://diplodatos.famaf.unc.edu.ar/


In [14]:
from util import load_datasets_unlabeled_test
# Obtenemos los datos en arreglos
train, dev, test = load_datasets_unlabeled_test()
X_train, y_train = train
X_dev, y_dev = dev

In [15]:
filename = 'subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff'
f = open(filename)
lines = f.readlines()
f.close()

In [16]:
words = []
for line in lines:
    sline = line.split()
    dline = dict([token.split('=') for token in sline if '=' in token])
    word = dline['word1']
    pol = dline['priorpolarity']
    if pol not in {'both', 'neutral'}:
        if pol in {'negative', 'weakneg'}:
            pol = 'NEG'
        else:
            pol = 'POS'
        words.append((word, pol))

word_dict = dict(words)

In [17]:
tkn = CountVectorizer().build_tokenizer()
def my_tkn(s):
    tokens = tkn(s)
    return [word_dict.get(token, token) for token in tokens]

In [23]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from util import print_eval

pipeline = Pipeline([
    ('vect', FeatureUnion([
        ('bow', CountVectorizer(binary=True)),
        ('pol', TfidfVectorizer(vocabulary=['POS', 'NEG'], tokenizer=my_tkn)),
    ])),
    ('clf', LogisticRegression(random_state=0)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	0.82

             precision    recall  f1-score   support

        neg       0.81      0.85      0.83        54
        pos       0.84      0.79      0.82        53

avg / total       0.82      0.82      0.82       107

[[46  8]
 [11 42]]


In [24]:
from sklearn.linear_model import LogisticRegressionCV
pipeline = Pipeline([
    ('vect', CountVectorizer(
        binary=True,
        min_df=3,
        max_df=0.90,
        ngram_range=(1, 5),
    )),
    ('clf', LogisticRegressionCV(random_state=0)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_dev, y_dev)

accuracy	0.91

             precision    recall  f1-score   support

        neg       0.88      0.94      0.91        54
        pos       0.94      0.87      0.90        53

avg / total       0.91      0.91      0.91       107

[[51  3]
 [ 7 46]]


# Probando el clasificador:
Probemos nuestro clasificador inicial

In [19]:
pipeline.predict(['this movie is truly good'])

array([1])

In [20]:
pipeline.predict(['this movie is truly bad'])

array([0])

In [21]:
from util import save_results

save_results('20',test)

TypeError: save_results() missing 1 required positional argument: 'labels'